In [19]:
import pandas as pd
from konlpy.tag import Mecab
import pickle

import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

#### 뉴스 데이터 로드 및 concat

In [20]:
data_1 = pd.read_csv("naver_news_data/정신건강정책_20211101_20211231.csv")
data_2 = pd.read_csv("naver_news_data/정신건강정책_20220101_20220228.csv")
data_3 = pd.read_csv("naver_news_data/정신건강정책_20220301_20220430.csv")
data_4 = pd.read_csv("naver_news_data/정신건강정책_20220501_20220630.csv")
data_5 = pd.read_csv("naver_news_data/정신건강정책_20220701_20220831.csv")
data_6 = pd.read_csv("naver_news_data/정신건강정책_20220930_20221031.csv")

In [21]:
news_df = pd.concat([data_1, data_2, data_3, data_4, data_5, data_6])

In [22]:
news_df.reset_index(inplace=True, drop=True)

In [23]:
data = news_df.copy()

In [24]:
with open(f"word_dictionary.pickle","rb") as fr:
    word_dictionary = pickle.load(fr)

In [25]:
temp = pd.DataFrame()
temp = data[data['content'].str.contains(word_dictionary[0], na=True)]
for i in range(len(word_dictionary)-1):
    temp = pd.concat([temp,data[data['content'].str.contains(word_dictionary[i+1], na=True)]])

In [26]:
temp.drop_duplicates(inplace=True)

In [27]:
data = temp

In [28]:
data.dropna(axis = 0, inplace=True)

In [29]:
data.reset_index(drop=True, inplace=True)

In [30]:
news_df = data

#### 정규표현식으로 한글과 공백을 제외한 문자 제거

In [31]:
news_df.content = news_df.content.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

D:\Anaconda3\envs\DL\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [32]:
for i in range(len(news_df)):
    news_df.content[i] = news_df.content[i].replace('\n', ' ')

In [33]:
news_df

,date,title,link,content
0,2021-12-29 14:54:03,"‘오미크론 대확산’에도…규제 풀고, 고삐 죄고 ‘제각각’",https://n.news.naver.com/mnews/article/022/000...,격리 단축 추가 규제 없어방역규제카드 보류 주 회 재택 명 이하 제한규제 더욱 바...
1,2021-12-29 13:01:00,미국·영국 풀고 프랑스·독일 죄고…오미크론 대응 제각각,https://n.news.naver.com/mnews/article/421/000...,확진자 연일 폭증 격리 단축 추가 규제 없어 주 회 재택 의무 사적 모임 명 이하 ...
2,2021-12-20 23:08:31,"[나이트포커스] ""네거티브 자제, 정책 대결로?""",https://n.news.naver.com/mnews/article/052/000...,진행 이광연 앵커 출연 최진봉 성공회대 신문방송학과 교수 장성철 대구가톨릭...
3,2021-12-16 14:54:56,[단독]유한기도 당한 심야조사…文정부 때가 朴 때보다 많았다,https://n.news.naver.com/mnews/article/025/000...,지난 일 극단적 선택을 했던 유한기 전 성남도시개발공사 개발사업본부장은 사망 사...
4,2021-12-12 10:06:01,"마성의 '단짠' 조합, 우리는 왜 이 맛에 열광할까",https://n.news.naver.com/mnews/article/648/000...,스토리단짠에 끌리는 과학적 이유히트상품 즐비일부선 음모론 제기도실제론 생존에 반드시...
...,...,...,...,...
2580,2022-02-15 10:09:04,&lt;인사&gt; 보건복지부,https://n.news.naver.com/mnews/article/014/000...,보건복지부 급 승진 재정운용담당관 김일열 장애인정책과장 최봉근 의료인력정책과...
2581,2022-02-15 10:04:17,[인사] 보건복지부,https://n.news.naver.com/mnews/article/001/001...,급 승진 재정운용담당관 김일열 장애인정책과장 최봉근 의료인력정책과장 차...
2582,2022-06-23 16:55:58,"박윤규 차관, AI·데이터기업에 첫 현장 발걸음…""걸림돌 해소하겠다""(종합)",https://n.news.naver.com/mnews/article/003/001...,기사내용 요약끝장 토론식격주로 민간과 함께 디지털 경제 강국 위한 정책방향 논의 계...
2583,2022-02-11 16:13:12,[인사] 보건복지부(종합),https://n.news.naver.com/mnews/article/001/001...,국장급 인사 인구정책실 보육정책관 전병왕 과장급 전보 정신건강관...


In [34]:
news_content = []

In [35]:
for i in range(len(news_df)):
    news_content.append(news_df.content[i])

In [36]:
news_df['token'] = str()

In [37]:
news_df

,date,title,link,content,token
0,2021-12-29 14:54:03,"‘오미크론 대확산’에도…규제 풀고, 고삐 죄고 ‘제각각’",https://n.news.naver.com/mnews/article/022/000...,격리 단축 추가 규제 없어방역규제카드 보류 주 회 재택 명 이하 제한규제 더욱 바...,
1,2021-12-29 13:01:00,미국·영국 풀고 프랑스·독일 죄고…오미크론 대응 제각각,https://n.news.naver.com/mnews/article/421/000...,확진자 연일 폭증 격리 단축 추가 규제 없어 주 회 재택 의무 사적 모임 명 이하 ...,
2,2021-12-20 23:08:31,"[나이트포커스] ""네거티브 자제, 정책 대결로?""",https://n.news.naver.com/mnews/article/052/000...,진행 이광연 앵커 출연 최진봉 성공회대 신문방송학과 교수 장성철 대구가톨릭...,
3,2021-12-16 14:54:56,[단독]유한기도 당한 심야조사…文정부 때가 朴 때보다 많았다,https://n.news.naver.com/mnews/article/025/000...,지난 일 극단적 선택을 했던 유한기 전 성남도시개발공사 개발사업본부장은 사망 사...,
4,2021-12-12 10:06:01,"마성의 '단짠' 조합, 우리는 왜 이 맛에 열광할까",https://n.news.naver.com/mnews/article/648/000...,스토리단짠에 끌리는 과학적 이유히트상품 즐비일부선 음모론 제기도실제론 생존에 반드시...,
...,...,...,...,...,...
2580,2022-02-15 10:09:04,&lt;인사&gt; 보건복지부,https://n.news.naver.com/mnews/article/014/000...,보건복지부 급 승진 재정운용담당관 김일열 장애인정책과장 최봉근 의료인력정책과...,
2581,2022-02-15 10:04:17,[인사] 보건복지부,https://n.news.naver.com/mnews/article/001/001...,급 승진 재정운용담당관 김일열 장애인정책과장 최봉근 의료인력정책과장 차...,
2582,2022-06-23 16:55:58,"박윤규 차관, AI·데이터기업에 첫 현장 발걸음…""걸림돌 해소하겠다""(종합)",https://n.news.naver.com/mnews/article/003/001...,기사내용 요약끝장 토론식격주로 민간과 함께 디지털 경제 강국 위한 정책방향 논의 계...,
2583,2022-02-11 16:13:12,[인사] 보건복지부(종합),https://n.news.naver.com/mnews/article/001/001...,국장급 인사 인구정책실 보육정책관 전병왕 과장급 전보 정신건강관...,


In [38]:
news_df['token_nv'] = ''

In [39]:
def tokenizing(df):
    tokenizer = Mecab('C:\mecab\mecab-ko-dic')
    specific_tags = ['NNG', 'NNP', 'VV', 'VA'] # 일반명사, 고유명사, 동사, 형용사
    stopwords = []
    for i in range(len(df)):
        temp = []
        df['token'][i] = tokenizer.pos(df['content'][i]) # pos tagging
        for j in range(len(df['token'][i])):
            
            if df['token'][i][j][1] in specific_tags:
                #temp = [word for word in temp if not word in stopwords] # 불용어 제거
                temp.append(df['token'][i][j][0])
        df['token_nv'][i] = temp
            
        
    

In [40]:
tokenizing(news_df)

In [41]:
news_df

,date,title,link,content,token,token_nv
0,2021-12-29 14:54:03,"‘오미크론 대확산’에도…규제 풀고, 고삐 죄고 ‘제각각’",https://n.news.naver.com/mnews/article/022/000...,격리 단축 추가 규제 없어방역규제카드 보류 주 회 재택 명 이하 제한규제 더욱 바...,"[(격리, NNG), (단축, NNG), (추가, NNG), (규제, NNG), (...","[격리, 단축, 추가, 규제, 없, 방역, 규제, 카드, 보, 재택, 이하, 제한,..."
1,2021-12-29 13:01:00,미국·영국 풀고 프랑스·독일 죄고…오미크론 대응 제각각,https://n.news.naver.com/mnews/article/421/000...,확진자 연일 폭증 격리 단축 추가 규제 없어 주 회 재택 의무 사적 모임 명 이하 ...,"[(확진, NNG), (자, XSN), (연일, NNG), (폭, NNG), (증,...","[확진, 연일, 폭, 증, 격리, 단축, 추가, 규제, 없, 재택, 의무, 사, 모..."
2,2021-12-20 23:08:31,"[나이트포커스] ""네거티브 자제, 정책 대결로?""",https://n.news.naver.com/mnews/article/052/000...,진행 이광연 앵커 출연 최진봉 성공회대 신문방송학과 교수 장성철 대구가톨릭...,"[(진행, NNG), (이광연, NNP), (앵커, NNG), (출연, NNG), ...","[진행, 이광연, 앵커, 출연, 최진봉, 성공회, 신문, 방송학, 교수, 장성철, ..."
3,2021-12-16 14:54:56,[단독]유한기도 당한 심야조사…文정부 때가 朴 때보다 많았다,https://n.news.naver.com/mnews/article/025/000...,지난 일 극단적 선택을 했던 유한기 전 성남도시개발공사 개발사업본부장은 사망 사...,"[(지난, VV+ETM), (일, NNG), (극단, NNG), (적, XSN), ...","[일, 극단, 선택, 유한기, 성남, 도시, 개발, 공사, 개발, 사업, 본부, 장..."
4,2021-12-12 10:06:01,"마성의 '단짠' 조합, 우리는 왜 이 맛에 열광할까",https://n.news.naver.com/mnews/article/648/000...,스토리단짠에 끌리는 과학적 이유히트상품 즐비일부선 음모론 제기도실제론 생존에 반드시...,"[(스토리, NNG), (단, MM), (짠, IC), (에, IC), (끌리, V...","[스토리, 끌리, 과학, 이유, 히트, 상품, 일부, 선, 음모론, 제기, 생존, ..."
...,...,...,...,...,...,...
2580,2022-02-15 10:09:04,&lt;인사&gt; 보건복지부,https://n.news.naver.com/mnews/article/014/000...,보건복지부 급 승진 재정운용담당관 김일열 장애인정책과장 최봉근 의료인력정책과...,"[(보건복지부, NNP), (급, NNG), (승진, NNG), (재정, NNG),...","[보건복지부, 급, 승진, 재정, 운용, 담당관, 김, 장애, 정책, 과장, 최봉근..."
2581,2022-02-15 10:04:17,[인사] 보건복지부,https://n.news.naver.com/mnews/article/001/001...,급 승진 재정운용담당관 김일열 장애인정책과장 최봉근 의료인력정책과장 차...,"[(급, NNG), (승진, NNG), (재정, NNG), (운용, NNG), (담...","[급, 승진, 재정, 운용, 담당관, 김, 장애, 정책, 과장, 최봉근, 의료, 인..."
2582,2022-06-23 16:55:58,"박윤규 차관, AI·데이터기업에 첫 현장 발걸음…""걸림돌 해소하겠다""(종합)",https://n.news.naver.com/mnews/article/003/001...,기사내용 요약끝장 토론식격주로 민간과 함께 디지털 경제 강국 위한 정책방향 논의 계...,"[(기사, NNG), (내용, NNG), (요약, NNG), (끝장, NNG), (...","[기사, 내용, 요약, 끝장, 토론, 격주, 민간, 디지털, 경제, 강국, 정책, ..."
2583,2022-02-11 16:13:12,[인사] 보건복지부(종합),https://n.news.naver.com/mnews/article/001/001...,국장급 인사 인구정책실 보육정책관 전병왕 과장급 전보 정신건강관...,"[(국장, NNG), (급, NNG), (인사, NNG), (인구, NNG), (정...","[국장, 급, 인사, 인구, 정책, 실, 보육, 정책, 관, 전병, 왕, 과장, 급..."


In [42]:
news_df.to_csv('news_tokenized.csv', encoding='utf-8-sig')